In [21]:
import nest_asyncio
nest_asyncio.apply()

from hddCRP.simulations import simulate_sessions
from hddCRP.modelBuilder import cdCRP
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

import arviz as az

import os
import sys

In [22]:



args = [0, 1, 1]
# if __name__ == "__main__":
#     for ii, arg in enumerate(sys.argv):
#         if(ii < len(args)):
#             args[ii] = arg

simulation_id = args[0];
min_blocks = args[1];
max_blocks = args[2];

run_range = range(0, 20)
results_directory = "Results/Simulations/"
OVERWRITE = False;

if(simulation_id == 0):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 1):
    alpha = 8
    different_context_weights = [0.5, 0.5];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 * n_blocks];
    num_subjects = lambda n_blocks : 1;
    num_responses = 3
elif(simulation_id == 2):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 20}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks : n_blocks;
    num_responses = 3
elif(simulation_id == 3):
    alpha = 8
    different_context_weights = [0.5, 0.5];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.25

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks :  n_blocks;
    num_responses = 3
elif(simulation_id == 4):
    alpha = 2
    different_context_weights = [0.8, 0.8];
    within_session_timescales  = {"A" : 50}
    between_session_timescales = {("A","A") : 2}
    repeat_bias_1_back = 0.75

    session_labels  = lambda n_blocks : ["A"] ;
    session_lengths = lambda n_blocks : [50 ];
    num_subjects = lambda n_blocks : n_blocks;
    num_responses = 3
else:
    raise NotImplementedError("No sim found")

repeat_bias_1_back = 1;
block_range = range(min_blocks, max_blocks+1)

if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)



In [23]:



for block_idx in block_range:
    print(f"BLOCK {block_idx}")
    fit_file = f"{results_directory}/sim_{simulation_id}_block_{block_idx}.pkl"
    fit_summary_file = f"{results_directory}/sim_summary_{simulation_id}_block_{block_idx}.pkl"
    if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
        simulation_fits = pd.DataFrame()
        simulation_fit_metrics = pd.DataFrame()
        for run_idx in run_range:
            print(f"BLOCK {block_idx} - RUN {run_idx}")
            sim_seed  = (simulation_id+1) * 10000 + run_idx*100
            stan_seed = (simulation_id+1) * 10000 + run_idx*100 + block_idx
            sim_rng = np.random.Generator(np.random.MT19937(sim_seed))
            
            seqs = [];
            subject_labels = [];
            session_labels_all = [];
            for jj in range(num_subjects(block_idx)):
                seqs_c = simulate_sessions(session_lengths=session_lengths(block_idx), session_labels=session_labels(block_idx), num_responses=num_responses, 
                                        alpha=alpha,
                                        different_context_weights=different_context_weights,
                                        within_session_timescales=within_session_timescales, between_session_timescales=between_session_timescales,
                                        repeat_bias_1_back=repeat_bias_1_back, rng=sim_rng)
                subject_labels += [jj] * len(seqs_c)
                session_labels_all += session_labels(block_idx)
                seqs += seqs_c;

            model = cdCRP(seqs, subject_labels=subject_labels, session_labels=session_labels_all);
            model.same_nback_depth = 0;

            model.build(random_seed=stan_seed);
            model.fit_model()

            map_fit = model.get_map()
            fit_df = model.fit.to_frame()
            fit_df["block"] = block_idx
            fit_df["run"]   = run_idx
            fit_df["simulation"]   = simulation_id
            summary_df = model.fit_summary()
            summary_df["block"] = block_idx
            summary_df["run"]   = run_idx
            summary_df["MAP"] = pd.Series(map_fit)
            summary_df["simulation"]   = simulation_id

            simulation_fit_metrics = pd.concat([simulation_fit_metrics,summary_df], copy=False)
            simulation_fits = pd.concat([simulation_fits,fit_df], copy=False)

        simulation_fits.to_pickle(fit_file)
        simulation_fit_metrics.to_pickle(fit_summary_file)
    else:
        print("Fit files found: not overriding")

BLOCK 1
BLOCK 1 - RUN 0
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 0.001196 seconds
  1000 transitions using 10 leapfrog steps per transition would take 11.96 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.002033 seconds
  1000 transitions using 10 leapfrog steps per transition would take 20.33 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.002806 seco

BLOCK 1 - RUN 1
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 0.002361 seconds
  1000 transitions using 10 leapfrog steps per transition would take 23.61 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.00225 seconds
  1000 transitions using 10 leapfrog steps per transition would take 22.5 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.002428 second

BLOCK 1 - RUN 2
Building...


...


Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  19% (1500/8

BLOCK 1 - RUN 18
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  14% (1100/8000)
Sampling:  16% (1300/8000)
Sampling:  19% (1500/8000)
Sampling:  21% (1700/8

BLOCK 1 - RUN 19
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/

In [24]:
simulation_fits
# observations are independent given a latent variable: a random measure  DP(alpha G)

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,timeconstant_within_session_A,context_similarity_depth_1,context_similarity_depth_2,block,run,simulation
draws,,,,,,,,,,,,,,
0,-53.740409,1.000000,0.559945,3.0,7.0,0.0,55.663210,4.979649,64.420483,0.906187,0.065587,1,0,0
1,-51.315848,0.992162,0.615045,2.0,3.0,0.0,51.544155,4.582011,78.181183,0.494169,0.457594,1,0,0
2,-52.260009,1.000000,0.577421,3.0,7.0,0.0,55.325758,3.996556,50.602283,0.900486,0.594418,1,0,0
3,-53.789497,0.967589,0.606054,3.0,7.0,0.0,54.770119,1.433693,41.881028,0.086715,0.904519,1,0,0
4,-51.448915,1.000000,0.559945,2.0,3.0,0.0,53.662739,4.877424,56.827167,0.470181,0.204976,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-55.660398,0.978609,0.523562,3.0,7.0,0.0,56.510100,1.066213,60.068599,0.069836,0.794268,1,19,0
3996,-52.342919,0.934672,0.496345,3.0,7.0,0.0,52.819264,4.565613,12.186830,0.271538,0.434255,1,19,0
3997,-52.285437,0.818628,0.588501,3.0,7.0,0.0,55.705928,6.608747,26.289943,0.728222,0.133495,1,19,0


In [25]:
model.to_dict()["subject_start_idx"]

array([ 1, 51])